In [ ]:
import os
import sys
import gc

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import pytorch_lightning as pl
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import albumentations as albu
from sklearn.model_selection import KFold, GroupKFold
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras import layers, models, optimizers, losses
from keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import StackingClassifier, VotingClassifier
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from scipy.signal import butter, lfilter
import tensorflow as tf
from keras.saving import load_model

import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from tensorflow.keras import layers, models, optimizers, losses
from keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import StackingClassifier, VotingClassifier
kl = losses.KLDivergence()
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from scipy.signal import butter, lfilter
import tensorflow as tf
from keras.saving import load_model
!pip install --no-index --find-links=/kaggle/input/tf-efficientnet-whl-files /kaggle/input/tf-efficientnet-whl-files/efficientnet-1.1.1-py3-none-any.whl

# Intro

First, sorry for the mess above, I pulled it from a notebook I have been working on different things to try and improve performance and I didn't clean up.

The purpose of this notebook is to determine which type of classes the initial network provided in the [EfficientNet](https://www.kaggle.com/code/cdeotte/efficientnetb0-starter-lb-0-43) starter notebook performs well on to guide what type of features and training methodologies would be valuebale for future solutions as part of a possible solution.  To do this, will be evaluating out of fold performance of each fold neural network.  It is noted that out of fold performance was not saved during train time, so I depending on an assumption that [GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) produces repeatable folds, though I did not validate this and only took this as possibly true based on some internet forum discussions, so this could not be actually true.  Due to this, actual performance of solution cannot be garunteed but comparisions of the network to itself may still be useful.

In [ ]:
# Load Training Data
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

In [ ]:
# Create Non-overlapping EEG ID Train Data
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

In [ ]:
print('We will use the following subset of raw EEG features:')
FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
FEAT2IDX = {x:y for x,y in zip(FEATS,range(len(FEATS)))}
print( list(FEATS) )

In [ ]:
# Read Train Specs
print('Reding Spectrograms...')
spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()
print('Train spectrograms read!')

In [ ]:
print('Reading EEG Spectrograms...')
all_eegs = np.load('/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()
print('EEG generated spectrograms read!')

In [ ]:
USE_KAGGLE_SPECTROGRAMS = True
USE_EEG_SPECTROGRAMS = True

In [ ]:
# USE MIXED PRECISION
MIX = True
if MIX:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
    print('Mixed precision enabled')
else:
    print('Using full precision')

In [ ]:
import albumentations as albu


TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=False, augment=False, mode='train',
                 specs = spectrograms, eeg_specs = all_eegs):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = int( np.ceil( len(self.data) / self.batch_size ) )
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        #if self.augment: X = self.__augment_batch(X)
        if self.augment: X, y = mixup_batch(X, y)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange( len(self.data) )
        if self.shuffle: np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((len(indexes),128,256,8),dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')
        img = np.ones((128,256),dtype='float32')

        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            if self.mode=='test':
                r = 0
            else:
                r = int( (row['min'] + row['max'])//4 )

            for k in range(4):
                # EXTRACT 300 ROWS OF SPECTROGRAM
                img = self.specs[row.spec_id][r:r+300,k*100:(k+1)*100].T
                #img = self.specs[row.spec_id][:,:].T
                # LOG TRANSFORM SPECTROGRAM
                img = np.clip(img,np.exp(-4),np.exp(8))
                img = np.log(img)

                # STANDARDIZE PER IMAGE
                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img-m)/(s+ep)
                img = np.nan_to_num(img, nan=0.0)

                # CROP TO 256 TIME STEPS
                X[j,14:-14,:,k] = img[:,:256] / 2.0

            # EEG SPECTROGRAMS - THIS IS NEW
            img = self.eeg_specs[row.eeg_id]
            # STANDARDIZE PER IMAGE
            #ep = 1e-6
            #m = np.nanmean(img.flatten())
            #s = np.nanstd(img.flatten())
            #img = (img-m)/(s+ep)
            #img = np.nan_to_num(img, nan=0.0)
            X[j,:,:,4:] = img

            if self.mode!='test':
                y[j,] = row[TARGETS]

        return X,y

    def __random_transform(self, img):
        composition = albu.Compose([
            albu.HorizontalFlip(p=0.5),
            albu.CoarseDropout(max_holes=8,max_height=16,max_width=8,fill_value=0,p=0.5),
            albu.GaussNoise(var_limit=(0.01, 0.05))
        ])
        return composition(image=img)['image']

    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i, ] = self.__random_transform(img_batch[i, ])
            img_batch[i, ] = augment(img_batch[i, ])
        return img_batch

In [ ]:
class KLDivergence:
    def __init__(self, reduction='auto', name='kl_divergence', fn=None):
        self.reduction = reduction
        self.name = name

    def __call__(self, y_true, y_pred):
        # Your Kullback-Leibler Divergence calculation logic here
        # Make sure to use self.reduction where applicable
        return 0

In [ ]:
def build_model():
    
    inp = tf.keras.Input(shape=(128,256,8))
    base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=None)
    
    # RESHAPE INPUT 128x256x8 => 512x512x3 MONOTONE IMAGE
    # KAGGLE SPECTROGRAMS
    x1 = [inp[:,:,:,i:i+1] for i in range(4)]
    x1 = tf.keras.layers.Concatenate(axis=1)(x1)
    # EEG SPECTROGRAMS
    x2 = [inp[:,:,:,i+4:i+5] for i in range(4)]
    x2 = tf.keras.layers.Concatenate(axis=1)(x2)
    # MAKE 512X512X3
    if USE_KAGGLE_SPECTROGRAMS & USE_EEG_SPECTROGRAMS:
        x = tf.keras.layers.Concatenate(axis=2)([x1,x2])
    elif USE_EEG_SPECTROGRAMS: x = x2
    else: x = x1
    x = tf.keras.layers.Concatenate(axis=3)([x,x,x])
    
    # OUTPUT
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)
        
    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()

    model.compile(loss=loss, optimizer = opt, metrics=['accuracy'] ) 
        
    return model

In [ ]:
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc
import efficientnet.tfkeras as efn

all_oof = []
all_true = []

gkf = GroupKFold(n_splits=5)

gc.collect()


for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):
    print(f'Evaluating Model {i+1}...')
    model = build_model()
    valid_gen = DataGenerator(train.iloc[valid_index], shuffle=False, batch_size=8, mode='valid', augment=False)
    model.load_weights(f'/kaggle/input/brain-efficientnet-models-v3-v4-v5/EffNet_v5_f{i}.h5')
    oof = model.predict(valid_gen, verbose=1)
    all_oof.append(oof)
    all_true.append(train.iloc[valid_index][TARGETS].values)
    
    del model, oof
    gc.collect()

all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)


# KL- Div Score

The KL-Div score is substantially better than shown in the notebook (0.57 CV, this is showing 0.45).  This is likely due to data leakage.  I do not intend to fix this in an attempt to preserve compute resources but in future trains the out of fold performance should be saved off for this analysis to be more useful.

In [ ]:
import sys
sys.path.append('/kaggle/input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for EfficientNetB0 =',cv)

# Confusion Matrix

This analysis is limited due to there being multiple possible diagnosis in each signal, this is ignored when using the argmax.  As such, the global performance isn't that valubale but comparing between classes can be.  We can see in these confusion matrixies that there is a problem detecting LRDA in the signals.  When this analysis is repeated with the PyTorch [ResNet](https://www.kaggle.com/code/yunsuxiaozi/hms-eegs-resnet34d-512-512-training-5-folds) model, it is seen this is not an issue for that network.  This helps explain why the [ensampling](https://www.kaggle.com/code/cody11null/quick-ensemble) of these two networks is so successful.  [This](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467000) discussion post is very helpful for understanding what is charactersitic of each type of classificaion.

**Possible Solutions**
* Class weighting
* Addition of features that help express information in the delta waves (some disagreement on what they are in literature. Some sort of range between 0.5 and 4 Hz is typical)
* Addition of features/models that help differentiate Lateral vs General activities

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


cm = confusion_matrix(np.argmax(all_true, axis=1), np.argmax(all_oof, axis=1))
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
class_labels = ['Seizure', 'LPD', 'GPD', 'LRDA', 'GRDA', 'Other']
plt.figure
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap='Blues', values_format='d')

plt.title('Confusion Matrix')
plt.show()

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent, display_labels=class_labels)
disp.plot(cmap='Blues', values_format='.2f')  # Use '.2f' for two decimal places

# ROC Curves

This analysis shows that the AUC of the KL-Divergence optimized solutions are in agreement.  This suggests that strong classificaion performance can be achieved without paying the price of over-confident wrong answers.

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# Initialize variables for ROC
fpr = dict()
tpr = dict()
roc_auc = dict()

# Compute ROC curve for each class
for i in range(6):
    # Use 'multilabel-indicator' to handle multiple labels for each sample
    fpr[i], tpr[i], _ = roc_curve((np.argmax(all_true, axis=1) == i).astype(int), all_oof[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure(figsize=(8, 8))

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple'])
for i, color in zip(range(6), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Class {class_labels[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (One-vs-All)')
plt.legend(loc="lower right")
plt.show()

# Final Conclusion

This network is struggling most with LRDA. This is the least frequent class, so adding more LRDA signals could held.  In the absence of more data, we should consider how to bring out important features to these signals. This signal classification is based on Delta-wave characteristics, which are generally definced as between 1-4 Hz.  May want to add specific filters focusing on pulling out the delta wave data.  Using Matlab's [buttord](https://www.mathworks.com/help/signal/ref/buttord.html), the order of the filter should be 4.  

This network would also benifit from features of General vs Lateralized events and Other events vs all other classificaiton events.

Ensampling with networks that have different weaknesses have been show to agregrate in better scores.